# Preperation of the data
I'm using the survey results of the Stack Overflow survey of 2018. The results of the survey mainly consists of categorical answers on questions like: 'Do you write code as a hobby?' or 'How many times do you exercise each week?'. Before I can use Machine Learning, I have to change all the results to numerical values. I'll also get rid of most of the missing values, because the average Machine Learning algorithm doesn't like these.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from scipy import stats

### Import the original Stack Overflow survey 2018 results

In [2]:
so_survey_original = pd.read_csv('./dataset/2018 Stack Overflow Survey Results.csv', low_memory=False)
so_survey_original.head(3)

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,4,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Copy the data frame to keep the prepped and original data frame seperately for comparison
so_survey = so_survey_original.copy(deep=True)

### Remove rows with missing job satisfaction
Our target feature is 'JobSatisfaction'. It isn't desired to have missing values for a target feature, because is doesn't conform to a classification.

In [4]:
mask = pd.isnull(so_survey['JobSatisfaction']) == False
so_survey = so_survey[mask].reset_index(drop=True)

print('rows before: %i, rows after: %i' % (so_survey_original.shape[0], so_survey.shape[0]))
so_survey

rows before: 98855, rows after: 69276


,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,4,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Full-stack developer,...,I don't typically exercise,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,35 - 44 years old,No,No,The survey was an appropriate length,Somewhat easy
4,7,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,...,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
5,8,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10 to 19 employees,Back-end developer;Database administrator;Fron...,...,1 - 2 times per week,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,18 - 24 years old,No,NaN,The survey was an appropriate length,Somewhat easy
6,9,Yes,Yes,United States,No,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Back-end developer;Front-end developer;Full-st...,...,I don't typically exercise,Male,Straight or heterosexual,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",White or of European descent,18 - 24 years old,No,No,The survey was an appropriate length,Somewhat easy
7,10,Yes,Yes,Nigeria,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10 to 19 employees,Designer;Front-end developer;QA or test developer,...,1 - 2 times per week,Female,NaN,Primary/elementary school,Black or of African descent,25 - 34 years old,No,NaN,The survey was too long,Somewhat difficult
8,11,Yes,Yes,United States,No,Employed full-time,Some college/university study without earning ...,Fine arts or performing arts (ex. graphic desi...,100 to 499 employees,"Back-end developer;C-suite executive (CEO, CTO...",...,I don't typically exercise,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,35 - 44 years old,Yes,No,The survey was an appropriate length,Very easy
9,17,Yes,No,Spain,No,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Information systems, information technology, o...","1,000 to 4,999 employees",Back-end developer;Database administrator;Mobi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove unimportant columns

#### 'Salary' and 'Salary Type'
The column 'ConvertedSalary' is the combination of the columns 'Salary' and 'SalaryType'. Both columns can therefore be removed.

In [5]:
so_survey.drop(so_survey[['Salary', 'SalaryType']], axis=1, inplace=True)

#### 'Respondent'
Respondent is an unique identifier, it won't add value for the model.

In [6]:
so_survey.drop(so_survey[['Respondent']], axis=1, inplace=True)

#### All columns with 'StackOverflow' and with 'Survey' in it
Questions with 'StackOverflow' and 'Survey' were used to gather feedback. These values won't have any correlation with job satisfaction.

In [7]:
columns = []
for column in so_survey.columns:
    if 'StackOverflow' in column or 'Survey' in column:
        columns.append(column)
so_survey.drop(so_survey[columns], axis=1, inplace=True)

#### 'TimeAfterBootcamp' and 'MilitaryUS'
Both columns have more than 80% missing data. Imputing statistical values instead won't result in accurate data.

In [8]:
so_survey.drop(so_survey[['TimeAfterBootcamp', 'MilitaryUS']], axis=1, inplace=True)

In [9]:
print('columns before: %i, columns after: %i' % (so_survey_original.shape[1], so_survey.shape[1]))

columns before: 129, columns after: 114


### Prepare values in list format
Some values in the dataset are denoted as 'Python;Java;C#' this list format needs to be splitted and converted to numerical values.

#### Find columns containing list format

In [10]:
def find_cols_containing_char(df, char):
    list_columns = []
    for col in df.columns:
        values = df[col].values
        if np.issubdtype(values.dtype, np.number) == False:
            for val in df[col].values:
                if char in str(val):
                    list_columns.append(col)
                    break
    return list_columns
list_columns = find_cols_containing_char(so_survey, ';')
list_columns

['DevType',
 'CommunicationTools',
 'EducationTypes',
 'SelfTaughtTypes',
 'HackathonReasons',
 'LanguageWorkedWith',
 'LanguageDesireNextYear',
 'DatabaseWorkedWith',
 'DatabaseDesireNextYear',
 'PlatformWorkedWith',
 'PlatformDesireNextYear',
 'FrameworkWorkedWith',
 'FrameworkDesireNextYear',
 'IDE',
 'Methodology',
 'VersionControl',
 'AdBlockerReasons',
 'AdsActions',
 'ErgonomicDevices',
 'Gender',
 'SexualOrientation',
 'RaceEthnicity']

#### Convert ';' seperated values to binarized representation
List values such as $'Python;Java;C#'$ can't be used as input for a Machine Learning algorithm. First, the value has to be numerical. Second, numerification of the ';' values as is will result a unique class for every unique list. It is instead needed to get a unique class for every language present in the list.<br><br>
To do this I use the *MultiLabelBinarizer* algorithm from the *sklearn.preprocessing* library. This algorithm converts values like $[['Python', 'Java', 'C#'], ['Python', 'C#']]$ to the following representation: $[[1, 1, 1], [1, 0, 1]]$ with the columns 'Python', 'Java' and 'C#'. This way the the values will be converted to a $0$ or a $1$ for each row, which is readable for a Machine Learning algorithm.

In [11]:
def binarize_list_columns(df, columns, sep=';'):
    df_copy = df.copy(deep=True)
    mlb = MultiLabelBinarizer()
    for col in columns:
        # Convert list formats like 'Python;Java;C#' to [Python, Java, C#]
        # If value is NaN, the value will be converted to []
        tranformed_vals = [x.split(';') if x is not np.nan else [] for x in df_copy[col].values]
        
        # Binerize the tranformed values
        binerized = mlb.fit_transform(tranformed_vals)

        # Add the binerized value to the existing data frame
        df_copy = df_copy.join(pd.DataFrame(binerized, columns=mlb.classes_), lsuffix='_left', rsuffix='_right')
        
        # Delete original column because it doesn't add value anymore
        del df_copy[col]
    return df_copy

# The column 'Student' will colide with the value 'Student' in the binarization process.
# Therefore the column 'Student' will be renamed to 'is_student'
so_survey.rename(columns={'Student': 'is_student'}, inplace=True)

# Start binarization of list columns
so_survey_bin = binarize_list_columns(so_survey, list_columns, ';')
print('columns before: %i, columns after: %i' % (so_survey.shape[1], so_survey_bin.shape[1]))
so_survey_bin.head(3)

columns before: 114, columns after: 410


,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Black or of African descent,East Asian,Hispanic or Latino/Latina,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian",South Asian,White or of European descent
0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,3-5 years,3-5 years,...,0,0,1,1,0,0,0,0,0,0
1,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",30 or more years,18-20 years,...,0,0,1,0,0,0,0,0,0,1
2,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,24-26 years,6-8 years,...,0,0,0,0,0,0,0,0,0,0


### Convert string categories to numerical categories

#### Find columns with string values

In [12]:
def find_object_cols(df):
    return [x for x in df.columns if df[x].dtype == object]
obj_cols = find_object_cols(so_survey_bin)
print('Number of columns with object dtype: %i' % (len(obj_cols)))

Number of columns with object dtype: 51


#### Convert string values to numerical values
The value containing string will be converted to numerical value. E.g. a column with the values 'yes' and 'no' will be mapped to 1 and 0.

In [13]:
def convert_to_numerical(df, columns):
    df_copy = df.copy(deep=True)
    maps = {}
    for col in columns:
        maps[col] = {key: value for value, key in enumerate(df[col].unique()) if key is not np.nan}
    df_copy.replace(maps, inplace=True)
    return df_copy, maps
                 
so_survey_conv, maps = convert_to_numerical(so_survey_bin, obj_cols)
print('Number of columns with object dtype: %i' % (len(find_object_cols(so_survey_conv))))
so_survey_conv.head(3)

Number of columns with object dtype: 0


,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Black or of African descent,East Asian,Hispanic or Latino/Latina,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian",South Asian,White or of European descent
0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,1,1,0,0,0,0,0,0
1,0,1,1,0.0,1,0.0,1.0,1.0,1.0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,1,2,0.0,1,1.0,2.0,0.0,2.0,2,...,0,0,0,0,0,0,0,0,0,0


### Export prepped data frame

In [14]:
so_survey_conv.to_csv('./dataset/so_survey_prepped.csv', index=False)

### Export used mappings

In [15]:
maps_inv = {}
for key in maps.keys():
    m = {}
    for k, v in maps[key].items():
        m[v] = k
    maps_inv[key] = m
mappings = pd.DataFrame(maps_inv)
mappings.head(3)

,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,Exercise,EducationParents,Age,Dependents
0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,3-5 years,3-5 years,...,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,3 - 4 times per week,"Bachelor’s degree (BA, BS, B.Eng., etc.)",25 - 34 years old,Yes
1,No,Yes,United Kingdom,"Yes, part-time",Employed full-time,Associate degree,"A natural science (ex. biology, chemistry, phy...","10,000 or more employees",30 or more years,18-20 years,...,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,NaN,Daily or almost every day,NaN,35 - 44 years old,NaN
2,NaN,NaN,United States,"Yes, full-time","Independent contractor, freelancer, or self-em...",Some college/university study without earning ...,"Computer science, computer engineering, or sof...",100 to 499 employees,24-26 years,6-8 years,...,NaN,NaN,NaN,NaN,NaN,3 - 4 times per week,NaN,Some college/university study without earning ...,NaN,No


In [16]:
mappings.to_csv('./dataset/so_survey_mappings.csv', index=False)

### Impute missing data
A lot of data is missing, this missing data can be imputed with statistical values (e.g. mean, mode, etc.)

In [17]:
so_survey_imp = so_survey_conv.copy(deep=True)

#### Impute missing values of numerical and categorical data types
Only the column 'ConvertedSalary' is of the numerical data type. We can impute the missing data of the numerical data type by using the mean of all values. All the other columns are of the categorical data type. We can impute the missing data of the categorical data type by using the mode of all values.

In [18]:
for col in so_survey_imp.columns:
    values = so_survey_imp[col].values
    stat_value = 0
    if col is 'ConvertedSalary':
        stat_value = values[~np.isnan(values)].mean()
    else:
        stat_value = stats.mode(values[~np.isnan(values)])[0][0]
    so_survey_imp[col].fillna(stat_value, inplace=True)

### Export imputed data frame

In [19]:
so_survey_imp.to_csv('./dataset/so_survey_prepped_imputed.csv', index=False)